In [3]:
import pandas as pd
import numpy as np

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
# plt.style.use('seaborn')             ggplot;
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'               #解决一些字体显示乱码问题
import warnings; warnings.simplefilter('ignore')       #忽略警告信息；

## 第一步，准备数据

In [5]:
data=pd.read_csv(r'D:\Baiducloud\12M8-202005\1MIN\EURUSD_M1.csv')
data.head()

,Time (UTC),Open,High,Low,Close,Volume
0,2010.01.01 00:00:00,1.43283,1.43283,1.43267,1.43267,74.3
1,2010.01.01 00:01:00,1.43268,1.43285,1.43268,1.43284,25.4
2,2010.01.01 00:02:00,1.43280,1.43280,1.43268,1.43280,30.9
3,2010.01.01 00:03:00,1.43280,1.43280,1.43268,1.43273,41.9
4,2010.01.01 00:04:00,1.43283,1.43283,1.43277,1.43277,53.8


In [13]:
data=data.drop(['Open','High','Low','Volume '],axis=1)

KeyError: "['Open' 'High' 'Low' 'Volume '] not found in axis"

In [14]:
data.head()

,Time (UTC),Close
0,2010.01.01 00:00:00,1.43267
1,2010.01.01 00:01:00,1.43284
2,2010.01.01 00:02:00,1.43280
3,2010.01.01 00:03:00,1.43273
4,2010.01.01 00:04:00,1.43277


In [20]:
data.rename(columns={'Time (UTC)':'date','Close':'price'},inplace=True)

In [21]:
data.head()

,date,price
0,2010.01.01 00:00:00,1.43267
1,2010.01.01 00:01:00,1.43284
2,2010.01.01 00:02:00,1.43280
3,2010.01.01 00:03:00,1.43273
4,2010.01.01 00:04:00,1.43277


In [23]:
data.set_index('date',drop=True,inplace=True)
data.head()

,price
date,
2010.01.01 00:00:00,1.43267
2010.01.01 00:01:00,1.43284
2010.01.01 00:02:00,1.43280
2010.01.01 00:03:00,1.43273
2010.01.01 00:04:00,1.43277


In [24]:
data.index=pd.to_datetime(data.index)
data.head()

,price
date,
2010-01-01 00:00:00,1.43267
2010-01-01 00:01:00,1.43284
2010-01-01 00:02:00,1.43280
2010-01-01 00:03:00,1.43273
2010-01-01 00:04:00,1.43277


In [25]:
#截取需要的数据，分析一下2019-11-29至2019-12-31的一段上涨行情，其实是满 H4的金叉行情
data_1=data['2019-11-29':'2020-1-1']
data_1.head()

,price
date,
2019-11-29 00:00:00,1.10112
2019-11-29 00:01:00,1.10109
2019-11-29 00:02:00,1.10107
2019-11-29 00:03:00,1.10108
2019-11-29 00:04:00,1.10108


In [26]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 32280 entries, 2019-11-29 00:00:00 to 2020-01-01 23:59:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   32280 non-null  float64
dtypes: float64(1)
memory usage: 504.4 KB


In [27]:
data_1.describe()

,price
count,32280.000000
mean,1.110770
std,0.004884
min,1.098210
25%,1.107870
50%,1.109520
75%,1.113610
max,1.123890


##### 数据整理完毕

## 第二步，计算回测情况

In [ ]:
## np.cumsum()累计求和
## np.argmax()返回一个numpy数组中最大值的索引值
## np.argmin()返回一个numpy数组中最小值的索引值

In [36]:
data_1['return']=np.log(data_1['price'] / data_1['price'].shift(1))
data_1.head()

,price,return
date,,
2019-11-29 00:00:00,1.10112,NaN
2019-11-29 00:01:00,1.10109,-0.000027
2019-11-29 00:02:00,1.10107,-0.000018
2019-11-29 00:03:00,1.10108,0.000009
2019-11-29 00:04:00,1.10108,0.000000


In [44]:
data_1.fillna(0,inplace=True)

In [45]:
data_1.iloc[0,1]

0.0

In [46]:
data_1.head()

,price,return
date,,
2019-11-29 00:00:00,1.10112,0.000000
2019-11-29 00:01:00,1.10109,-0.000027
2019-11-29 00:02:00,1.10107,-0.000018
2019-11-29 00:03:00,1.10108,0.000009
2019-11-29 00:04:00,1.10108,0.000000


In [48]:
#找到这段时间的最高点和最低点
data_1['cumsum']=data_1['return'].cumsum().apply(np.exp)

In [49]:
data_1.head()

,price,return,cumsum
date,,,
2019-11-29 00:00:00,1.10112,0.000000,1.000000
2019-11-29 00:01:00,1.10109,-0.000027,0.999973
2019-11-29 00:02:00,1.10107,-0.000018,0.999955
2019-11-29 00:03:00,1.10108,0.000009,0.999964
2019-11-29 00:04:00,1.10108,0.000000,0.999964


从哪些角度去思考，从而抓住特征  
1.空间，产生个几个高点；什么是高点，所谓的高点是一个一段时间内的price最大值；  
2.该最高点维持的时间有多久；  
3.跨度比较长的这段时间呢，价格经历了怎么样的曲折，又产生了怎么样的回撤；

#### 1.找出该时段内的最高点和最低点

In [53]:
cumsum_min=data_1['cumsum'].min()    #最小值是多少
round(cumsum_min,5)

0.99736

In [54]:
cumsum_max=data_1['cumsum'].max()   #最大值是多少
round(cumsum_max,5)

1.02068

In [ ]:
i=